In [2]:
import pickle
from flask import Flask, request, jsonify
import pandas as pd
import mysql.connector


In [ ]:
connection = mysql.connector.connect(host = "localhost", user = "root", password = "*****", database = "Home_Scout")


app = Flask(__name__)

def process_data(data):
    df = pd.DataFrame(data, index=[0])
        
    with open('../Pickle/encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)

    cols = ["city", "state", "zip_code", "status", "sold"]
    encoded_user_data = encoder.transform(df[cols])

    encoded_user_data['bed'] = df['bed']
    encoded_user_data['bath'] = df['bath']
    encoded_user_data['acre_lot'] = df['acre_lot']
    encoded_user_data['house_size'] = df['house_size']
    encoded_user_data.drop(columns=['zip_code_A>'], inplace=True)

    return encoded_user_data

@app.route('/Add_User', methods=['POST'])
def Add_User():
    name = request.get_json()
    cursor = connection.cursor()
    QI = "INSERT INTO Users (Username) VALUES (%s);"
    val = (name,)
    cursor.execute(QI, val)
    connection.commit()
    cursor.close()
    print("User Added")
    
    return 'User Added'

@app.route('/Get_Users', methods=['GET'])
def Get_Users():
    Users = []
    QE = "SELECT Username FROM Users;"
    cursor = connection.cursor()
    cursor.execute(QE,)
    values = cursor.fetchall()
    for x in values:
        for y in x:
            Users.append(y)
    cursor.close()
    print("User list has been retrieved")
    return Users

@app.route('/Get_Predictions', methods=['GET'])
def Get_Predictions():
    QP = "SELECT * FROM Predictions;"
    cursor = connection.cursor()
    cursor.execute(QP,)
    Preds = cursor.fetchall()
    cursor.close()
    print("Prediction list has been retrieved")
    return Preds

@app.route('/Get_Reports', methods=['GET'])
def Get_Reports():
    QP = "SELECT * FROM Reported;"
    cursor = connection.cursor()
    cursor.execute(QP,)
    Reports = cursor.fetchall()
    cursor.close()
    print("Reports list has been retrieved")
    return Reports

@app.route('/View_Past_Predictions', methods=['GET'])
def View_Past_Predictions():
    Username = request.get_json()
    QS = "Select bed, bath, hsize, acre, city, state, zip, price FROM Predictions WHERE Username = %s"
    val = (Username,)
    cursor = connection.cursor()
    cursor.execute(QS, val)
    values = cursor.fetchall()
    cursor.close()
    print("Reported list has been retrieved")
    return values

@app.route('/predict', methods=['POST'])
def predict():
    RF = pickle.load(open('../Pickle/RandomForest.pkl', 'rb'))

    data = request.get_json()
    data = process_data(data)
    df = pd.DataFrame.from_dict(data)
    prediction = RF.predict(df)
    print("Label Predcted")
    return jsonify(prediction.tolist())

@app.route('/Add_Prediction', methods=['POST'])
def Add_Prediction():
    data = request.get_json()
    val = tuple(data)
    cursor = connection.cursor()
    QI = "INSERT INTO Predictions (Username, status, bed, bath, acre, city, state, zip, hsize, sold, price) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"       
    cursor.execute(QI, val)
    connection.commit()
    cursor.close()
    print('Prediction Added')
    return 'Prediction Added'

@app.route('/Add_Train', methods=['POST'])
def Add_Train():
    data = request.get_json()
    Val = tuple(data)
    cursor = connection.cursor()
    QT = "INSERT INTO Train (status, bed, bath, acre_lot, city, state, zip_code, house_size, sold, binned_label) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    cursor.execute(QT, Val)
    connection.commit()
    cursor.close()
    print('Data point Added to train set')
    return 'Data point Added to train'

@app.route('/Add_Reported', methods=['POST'])
def Add_Reported():
    data = request.get_json()
    Val = tuple(data)
    cursor = connection.cursor()
    QI = "INSERT INTO Reported (Username, bed, bath, hsize, acre, city, state, zip, price) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    cursor.execute(QI, Val)
    connection.commit()
    cursor.close()
    print('Data point reported')
    return 'Data point reported'

try:
    if __name__ == '__main__':
        app.run(port=5000, debug= False)
except Exception as e:
    print('Error:', e)
